In [1]:
import pandas as pd
import requests
import json
import logging
from bs4 import BeautifulSoup
import time

In [2]:
# GET TOP TOURISM COUNTRIES
df_tourism = pd.read_csv('./data/most_traveled_countries.csv')
# SOURCE: http://worldpopulationreview.com/countries/most-visited-countries/
# df_tourism.head()

top20 = list(df_tourism.sort_values(by='touristArrivals', ascending=False)['name'][:20])
print(top20)

['France', 'United States', 'Spain', 'China', 'Italy', 'United Kingdom', 'Germany', 'Mexico', 'Thailand', 'Turkey', 'Austria', 'Malaysia', 'Hong Kong', 'Greece', 'Russia', 'Japan', 'Canada', 'Portugal', 'Saudi Arabia', 'Poland']


In [3]:
# GET COUNTRIES
country_names = [c_name for c_name in pd.read_csv('./data/country_names.csv')['name']]
# SOURCES: http://worldpopulationreview.com/countries/most-visited-countries/
    # & https://www.travel-advisory.info/api


In [4]:
len(country_names)

178

In [5]:
# GET ALL URLS FOR 20 COUNTRIES
articles = []
for country in country_names:
#     print(f'Searching {country}...')
    for n in range(1, 4):
        try:
            url_base = f'https://nerdnomads.com/{country}'
            res_ = requests.get(url_base)
            soup_ = BeautifulSoup(res_.content, 'lxml')
            time.sleep(1)

            for link in soup_.find_all(attrs={'class': 'grid-title'}):
                article = {}
                article['href'] = link.find('a').attrs['href']
                article['title'] = link.text.strip()
                article['country'] = country
                articles.append(article)
        except:
            print(f'Error with {country} at page {n}.')

df = pd.DataFrame(articles)
# df.to_csv('./data/all_articles.csv')
print('Shape of dataframe:', df.shape)
df.head()

Shape of dataframe: (132, 3)


,href,title,country
0,https://nerdnomads.com/komodo-national-park-to...,Eden Found? – Komodo National Park Day 3,Indonesia
1,https://nerdnomads.com/komodo-national-park-to...,In Search Of The Komodo Dragon – Komodo Nation...,Indonesia
2,https://nerdnomads.com/komodo-national-park-to...,The Adventure Begins – Komodo National Park Day 1,Indonesia
3,https://nerdnomads.com/hindu-masterpiece-pramb...,The Hindu Masterpiece – Prambanan Temple,Indonesia
4,https://nerdnomads.com/eat-pray-love-ubud-bali,"Eat, Pray, Love In Ubud, Bali",Indonesia


In [6]:
content_filed = []
for url in df['href']:
    res_post = requests.get(url)
    soup_post = BeautifulSoup(res_post.content, 'lxml')

    paragraphs = soup_post.find_all('p', attrs={'class': None})

    text = ' '.join([paragraph.text.replace('\n', '') for paragraph in paragraphs])
    content_filed.append(text)
    
    time.sleep(1)
df['text'] = content_filed

In [7]:
df.tail()

,href,title,country,text
127,https://nerdnomads.com/golden-mount-wat-saket-...,Bangkok´s Temple In The Sky – Golden Mount (Wa...,Thailand,The sound of chanting monks and bells ring sof...
128,https://nerdnomads.com/thai-boxing-bangkok,Experiencing Thai Boxing in Bangkok,Thailand,My pulse is hammering in my ears. I’m breathin...
129,https://nerdnomads.com/where-to-stay-in-bangkok,Where To Stay In Bangkok – Our Favourite Areas...,Thailand,"Being a popular tourist destination, Bangkok i..."
130,https://nerdnomads.com/what-to-do-in-bangkok,What To Do In Bangkok – A 3 Day Bangkok Itinerary,Thailand,"Bangkok or Krung Thep, “City of Angels“, is li..."
131,https://nerdnomads.com/lumphini-park,"A Green Oasis – Lumphini Park, Bangkok",Thailand,"Lumphini Park is the biggest park in Bangkok, ..."


In [8]:
df.to_csv('./data/articles_nerdnomads.csv', index=False)